In [ ]:
import numpy as np
import random
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


# Loading and Preprocessing the Dataset

In [ ]:
# Define the dataset path
dataset_path = '/content/Alzheimer_MRI_4_classes_dataset'

# Image size for resizing
image_size = (128, 128)

# Function to load images and labels
def load_mri_data(path):
    X = []
    y = []
    classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

    # Loop through each class folder
    for label in classes:
        folder_path = os.path.join(path, label)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            try:
                # Load and resize image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, image_size)
                X.append(image)
                y.append(label)
            except Exception as e:
                print(f"Error loading image {image_name}: {e}")

    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Normalize image data
    X = X / 255.0
    X = X.reshape(-1, image_size[0] * image_size[1])

    # Encode labels to integers
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    return X, y, label_encoder

# Load the data
X, y, label_encoder = load_mri_data(dataset_path)
print(f"Data loaded. Shape: {X.shape}, Labels: {len(set(y))}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Data loaded. Shape: (3785, 16384), Labels: 4


# Creating the MLP Model

In [ ]:
from tensorflow.keras.layers import Input

def create_mlp_model(input_shape, hidden_layers, units):
    model = Sequential()

    # Adding Input layer explicitly
    model.add(Input(shape=(input_shape,)))
    model.add(Flatten())

    # Adding hidden layers
    for _ in range(hidden_layers):
        model.add(Dense(units, activation='relu'))

    # Output layer for 4 classes
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# Implementing the Fitness Function

In [ ]:
# Fitness evaluation function
def fitness(individual, X_train, y_train, X_test, y_test):
    try:
        model = create_mlp_model(X_train.shape[1], hidden_layers=individual['hidden_layers'], units=individual['units'])
        model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

        # Predict and calculate accuracy
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy
    except Exception as e:
        print(f"Error in fitness evaluation: {e}")
        return 0


# Genetic Algorithm Components (Population, Crossover, Mutation)

In [ ]:
def generate_population(size):
    return [{'hidden_layers': random.randint(1, 5), 'units': random.choice([16, 32, 64, 128, 256])} for _ in range(size)]

def crossover(parent1, parent2):
    child = {}
    child['hidden_layers'] = random.choice([parent1['hidden_layers'], parent2['hidden_layers']])
    child['units'] = random.choice([parent1['units'], parent2['units']])
    return child

def mutate(individual):
    if random.random() < 0.1:
        individual['hidden_layers'] = random.randint(1, 5)
    if random.random() < 0.1:
        individual['units'] = random.choice([16, 32, 64, 128, 256])
    return individual


#  Implementing the Genetic Algorithm

In [ ]:
def genetic_algorithm(X_train, y_train, X_test, y_test, population_size=10, generations=10):
    population = generate_population(population_size)
    best_fitness = 0
    best_individual = None

    for generation in range(generations):
        population = sorted(population, key=lambda x: fitness(x, X_train, y_train, X_test, y_test), reverse=True)
        next_gen = population[:2]  # Elitism

        while len(next_gen) < population_size:
            parent1, parent2 = random.sample(population[:4], 2)
            child = crossover(parent1, parent2)
            next_gen.append(mutate(child))

        population = next_gen
        best_candidate = population[0]
        best_candidate_fitness = fitness(best_candidate, X_train, y_train, X_test, y_test)

        print(f"Generation {generation}: Best Hidden Layers={best_candidate['hidden_layers']}, Units={best_candidate['units']}, Fitness={best_candidate_fitness:.4f}")

        if best_candidate_fitness > best_fitness:
            best_fitness = best_candidate_fitness
            best_individual = best_candidate

    return best_individual, best_fitness


# Training and Evaluating the Initial MLP Model

In [ ]:
print("Training Initial MLP Model:")
model = create_mlp_model(X_train.shape[1], hidden_layers=2, units=64)
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate initial model
y_pred = np.argmax(model.predict(X_test), axis=-1)
initial_accuracy = accuracy_score(y_test, y_pred)
print(f"Initial Model Accuracy: {initial_accuracy:.4f}")


Training Initial MLP Model:
Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4834 - loss: 1.7129
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6053 - loss: 0.9057
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6238 - loss: 0.8841
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6999 - loss: 0.7095
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6940 - loss: 0.7331
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7046 - loss: 0.7031
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.7625 - loss: 0.5839
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7857 - loss: 0.5369
Epoch 9/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7742 - loss: 0.5760
Epoch 10/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8112 - loss: 0.4572
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Initial Model Accuracy: 0.8058


# Running Genetic Algorithm for Optimization

In [ ]:
print("Running Genetic Algorithm for Optimization:")
best_individual, best_fitness = genetic_algorithm(X_train, y_train, X_test, y_test)
print(f"\nBest Configuration Found: Hidden Layers={best_individual['hidden_layers']}, Units={best_individual['units']}, Fitness={best_fitness:.4f}")


Running Genetic Algorithm for Optimization:
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Generation 0: Best Hidden Layers=1, Units=64, Fitness=0.7041
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
Generation 1: Best Hidden Layers

# Training MLP with Optimized Hyperparameters

In [ ]:
print("Training Optimized MLP Model:")
model = create_mlp_model(X_train.shape[1], hidden_layers=best_individual['hidden_layers'], units=best_individual['units'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate optimized model
y_pred = np.argmax(model.predict(X_test), axis=-1)
optimized_accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Model Accuracy: {optimized_accuracy:.4f}")


Training Optimized MLP Model:
Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5076 - loss: 1.1581
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6312 - loss: 0.8778
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6382 - loss: 0.8477
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6445 - loss: 0.8065
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6873 - loss: 0.7342
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6586 - loss: 0.7548
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7276 - loss: 0.6457
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.6950 - loss: 0.6957
Epoch 9/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7625 - loss: 0.5620
Epoch 10/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7819 - loss: 0.5111
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Optimized Model Accuracy: 0.8045
